**Nome**: Gabriel Henrique Przytocki <br>
**Data**: 20/09/2021

In [1]:
# Imports necessários
import pandas as pd
import numpy as np
import string
from functools import reduce
import sqlalchemy as db
from bs4 import BeautifulSoup
import requests
from threading import Thread

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

# Concluídos
- [X] 1 - Imprima seu nome completo utilizando Python
- [X] 2 - Criação de um novo arquivo processado
- [X] 3 - Ler arquivo e imprimir informações na tela
    - [X] 3.1
    - [X] 3.2
- [X] 4 - Lógica de programação
- [ ] 5 - Web Scraping
    - [ ] 5.1
    - [X] 5.2
- [X] 6 - Gravar arquivos em uma tabela de banco de dados
- [X] 7 - Query 1
- [X] 8 - Query 2
- [X] 9 - Query 3
- [X] 10 - Query 4

### Carregando os dados

In [3]:
# Subir o arquivo no ambiente local e carregar completamente
path = "receitas_candidatos_2014_brasil_english.csv"
df = pd.read_csv(path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head(2)

,Unnamed: 0,id_election,cat_election_description,dat_donation_date_time,id_accountant_cnpj,id_candidate_seq,cat_election_state,cat_party,id_candidate_num,cat_political_office,cat_candidate_name,id_candidate_cpf,id_receipt_num,id_document_num,id_donator_cpf_cnpj,cat_donator_name,cat_donator_name2,cat_donator_state,id_donator_party,id_donator_number,id_donator_economic_sector,cat_donator_economic_sector,dat_donation_date,num_donation_ammount,cat_donation_type,cat_donation_source,cat_donation_method,cat_donation_description,id_original_donator_cpf_cnpj,cat_original_donator_name,cat_original_donator_type,cat_original_donator_economic_sector,cat_original_donator_name2
0,0,143,Eleições Gerais 2014,09/07/201617:15:01,20578194000114,90000000637,GO,PSDB,45,Governador,MARCONI FERREIRA PERILLO JUNIOR,3553821809,000450300000GO000064,20141550000,1.256007e+12,GOVESA GOIANIA VEICULOS SA,GOVESA GOIANIA VEICULOS SA,NaN,NaN,45,4511101.0,"Comércio a varejo de automóveis, camionetas e ...",02/10/201400:00:00,145000.0,Recursos de pessoas jurídicas,Nao especificado,Transferência eletrônica,NaN,NaN,NaN,NaN,NaN,NaN
1,1,143,Eleições Gerais 2014,09/07/201617:15:01,20571797000194,90000000725,GO,PSD,5555,Deputado Federal,JOSE MARIO SCHREINER,41877004987,055550600000GO000090,553338000003336,1.256007e+12,GOVESA GOIANIA VEICULOS S/A,GOVESA GOIANIA VEICULOS SA,NaN,NaN,5555,4511101.0,"Comércio a varejo de automóveis, camionetas e ...",16/10/201400:00:00,100000.0,Recursos de pessoas jurídicas,Nao especificado,Transferência eletrônica,NaN,NaN,NaN,NaN,NaN,NaN


# Teste Python

1. Para responder as perguntas utilize células entre uma e outra pergunta.

2. Pra responder as perguntas utilizar o *dataset* __[Electoral Donations in Brazil](https://www.kaggle.com/felipeleiteantunes/electoral-donations-brazil2014)__ do Kaggle
    
3. O *download* tem um arquivo zip com vários arquivos, porém vamos utilizar somente o arquivo: ``receitas_candidatos_2014_brasil_english.csv``

    **Obs**: No código considere que o arquivo está no mesmo diretório onde irá executar o seu código python.


## 1. Imprima seu nome completo utilizando Python

In [5]:
print("Gabriel Henrique Przytocki")

Gabriel Henrique Przytocki


In [6]:
clfa = DecisionTreeClassifier()
clfa.fit(np.reshape(list(range(1,27)),(-1,1)), list(string.ascii_lowercase))
scaler = MinMaxScaler(feature_range=(1,26))

X_train = [-0.76,-1,-0.96,-0.32,-0.68,-0.84,-0.56,-0.72,-0.84,-0.48,-0.32,
           -0.68,-0.36,-0.2,-0.84,-0.4,-0.32,0,-0.04,-0.24,-0.44,-0.92,-0.6,-0.68]

X_test = scaler.fit_transform(np.reshape(X_train,(-1,1)))
pred = clfa.predict(X_test)

name = reduce(lambda x, y: x + y, np.insert(pred,[7,15]," ")).title()
print(name)

Gabriel Henrique Przytocki


## 2. Criação de um novo arquivo processado

1. Formato do Arquivo: "``parquet``"
2. Nome "``doacoes_por_partido.parquet``"
3. Colunas:
    - ``Partido``
    - ``Estado``
    - ``ValorDoado``: Soma de valor das doações agrupando partido e estado
    - ``ValorDoadoPorEstado``: Soma de todas as doações agrupando somente o estado
    - ``Percentual``: Razão entre o Valor doado para cada partido pelo valor doado total do estado

    Com base nas informações acima construir um codigo Python que processe os dados do arquivo "``receitas_candidatos_2014_brasil_english.csv``" e gere um novo arquivo somente com as colunas desejadas.

In [7]:
df_parquet = pd.DataFrame()

# Começando pelas colunas mais simples
df_parquet['Partido'] = df['cat_party']
df_parquet['Estado']  = df['cat_election_state']

"""Para as colunas seguintes, agrupei e agreguei os dados conforme os itens,
   em seguida iterei sobre os elementos aplicando uma função que retorna, 
   com base nos atributos da linha, os valores correspondentes nos conjuntos
   agrupados de dados. Por exemplo: agrupamos por total do estado em doações,
   logo iteramos sob a base original, e o proponente é do PR, logo aplicamos
   a função que busca no conjunto agregado, acessa o index 'PR' e obtém o
   valor sumarizado, retornando para preenchimento daquela posição. Isso se
   demonstrou necessário, pois tinhamos o index em formatos distintos após
   este ser agrupado, então desta forma mantemos o index original e não
   perdemos informação"""

# ValorDoado
valor_doado_partido_estado = df.groupby(['cat_party', 'cat_election_state']) \
                               .agg('sum')['num_donation_ammount']

def column_groupby_party_and_state(row: pd.Series) -> float:
    """
    Função que recebe um linha (pd.Series) de um DataFrame como 
    parâmetro, obtendo o partido e estado correspondente à essa 
    linha, retornando o valore agregado correspondentes em 
    outro DataFrame.

    Params:
        row (pd.Series): linha do DataFrame
    
    Returns:
        Novo valor a ser adicionado em outra coluna agrupada
    """
    partido = row['cat_party']
    estado = row['cat_election_state']
    return valor_doado_partido_estado[(f'{partido}', f'{estado}')]

df_parquet['ValorDoado'] = df.apply(column_groupby_party_and_state, 
                                    axis=1)

In [8]:
# ValorDoadoPorEstado
soma_estado = df.groupby('cat_election_state') \
                .agg('sum')['num_donation_ammount']

def column_groupby_state(row: pd.Series) -> float:
    """
    Função que recebe um linha (pd.Series) de um DataFrame como 
    parâmetro, obtendo o estado correspondente à essa 
    linha, retornando o valore agregado correspondentes em 
    outro DataFrame.

    Params:
        row (pd.Series): linha do DataFrame
    
    Returns:
        Novo valor a ser adicionado em outra coluna agrupada
    """
    estado = row['cat_election_state']
    return soma_estado[estado]

df_parquet['ValorDoadoPorEstado'] = df.apply(column_groupby_state,
                                             axis=1)

In [9]:
# Percentual
soma_partido = df.groupby('cat_party') \
                 .agg('sum')['num_donation_ammount']

def column_groupby_ratio(row: pd.Series) -> float:
    """
    Função que recebe um linha (pd.Series) de um DataFrame como 
    parâmetro, obtendo o partido e estado correspondente à essa 
    linha, retornando a razão entre a soma do partido relacionado 
    à linha e a soma do estado. É a razão entre o recebido pelo
    partido e o total do estado.

    Params:
        row (pd.Series): linha do DataFrame
    
    Returns:
        Novo valor correspondente à razão dos totais a ser adicionado 
        em outra coluna
    """
    partido = row['cat_party']
    estado = row['cat_election_state']
    return soma_partido[partido] / soma_estado[estado]

df_parquet['Percentual'] = df.apply(column_groupby_ratio,
                                    axis=1)

In [10]:
df_parquet.head()

,Partido,Estado,ValorDoado,ValorDoadoPorEstado,Percentual
0,PSDB,GO,69779749.09,2.047315e+08,3.600118
1,PSD,GO,23586590.68,2.047315e+08,1.186711
2,PSB,PI,10844707.44,6.165839e+07,4.948509
3,PMDB,PI,10854411.23,6.165839e+07,10.963549
4,PMDB,PI,10854411.23,6.165839e+07,10.963549


In [11]:
# Criando o arquivo no diretório atual
df_parquet.to_parquet('receitas_candidatos_2014_brasil_english.parquet')

## 3. Ler arquivo e imprimir informações na tela

Com base no arquivo gerado no item anterior "``doacoes_por_partido.parquet``" criar dois codigos em Python que respondam as seguintes perguntas:

- **3.1 - Imprimir uma lista utilizando Python** 
    - Criar uma lista com Estado, Partido, Percentual de doação
    - Filtrar somente os partidos com maiores doações em cada estado
    - Ordenar a lista pelo percentual de doação decrescente.
<br><br>
- **3.2 - Imprimir uma lista utilizando Python** 
    - Imprimir todas as colunas do arquivo
    - Filtrar pelos top 20 maiores valores na coluna ``ValorDoado``

In [12]:
# Carregando os dados do arquivo parquet
pq = pd.read_parquet('receitas_candidatos_2014_brasil_english.parquet')
pq.head()

,Partido,Estado,ValorDoado,ValorDoadoPorEstado,Percentual
0,PSDB,GO,69779749.09,2.047315e+08,3.600118
1,PSD,GO,23586590.68,2.047315e+08,1.186711
2,PSB,PI,10844707.44,6.165839e+07,4.948509
3,PMDB,PI,10854411.23,6.165839e+07,10.963549
4,PMDB,PI,10854411.23,6.165839e+07,10.963549


In [13]:
# Desenvolva a Resposta do item 3.1 Aqui
pq_groupby_partido_estado = pq.groupby('Estado') \
                            .agg('max') \
                            .sort_values(by='Percentual', ascending=False)

pq_groupby_partido_estado.drop(['ValorDoado','ValorDoadoPorEstado'], 
                               axis=1, 
                               inplace=True)

pq_groupby_partido_estado.head()                        

,Partido,Percentual
Estado,,
AP,SD,39.536587
SE,SD,35.481375
AC,SD,35.249656
RR,SD,23.315765
PI,SD,15.571411


In [14]:
# Desenvolva a Resposta do item 3.2 Aqui

"""Precisamos reover os itens duplicados, de maneira a ver
   os 20 maiores valores de 'ValorDoado', pois temos registros
   iguais nas 20 primeiras posições"""

pq_sorted = pq.drop_duplicates()
pq_sorted.sort_values(by='ValorDoado', ascending=False)[:20]

,Partido,Estado,ValorDoado,ValorDoadoPorEstado,Percentual
317,PT,BR,3.504934e+08,6.450946e+08,1.488322
2914,PSDB,BR,2.268581e+08,6.450946e+08,1.142558
22,PT,SP,1.438055e+08,5.905367e+08,1.625823
51,PSDB,SP,1.273637e+08,5.905367e+08,1.248115
340,PMDB,RJ,1.148136e+08,3.129615e+08,2.159993
348,PT,MG,9.375110e+07,4.223866e+08,2.273055
349,PSDB,MG,9.372964e+07,4.223866e+08,1.744984
0,PSDB,GO,6.977975e+07,2.047315e+08,3.600118
6115,PSB,BR,6.157121e+07,6.450946e+08,0.472980
244,PMDB,CE,5.857277e+07,1.696443e+08,3.984778


## 4. Lógica de programação 

1. Desenvolva utilizando **estruturas de repetição**, uma função que receba um vetor e um inteiro por parâmetro, e realize um *shift* no vetor do tamanho do número inteiro recebido.
        
- O tamanho do *shift* deverá aceitar números negativos também. O sinal do tamanho servirá para definir a direção do *shift*.
    - Números positivos indicam o *shift* da direita para a esquerda.
    - Números negativos indicam o *shift* da esquerda para a direita.
<br><br>
- **Não serão aceitas respostas do tipo:**
    - "``return vetor[-tamanho:] + vetor[:-tamanho]``"

In [15]:
# Desenvolva a Resposta do item 4 aqui

"""A ideia principal é criar um novo vetor, este que receberá n 
   posições deslocadas (para esquerda ou direita), portanto 
   podemos criar sequencialemnte cada elemento do novo vetor
   pegando o elemento atual do antigo vetor +- n deslocamentos,
   e esse será o elemento do novo vetor. Entretanto, para os casos
   negativos de n, essa ideia não funciona, bem como valores de 
   n > len(vetor). O resto da divisão (%) funciona, pois ignora as
   "repetições" de len(vetor), obtendo apenas o resto, ou, em outras 
   palavras, n. Para os casos negativos, funciona como um complemento
   equivalente, e positivo, de len(vetor). Em outras palavras, nos
   fornece um tamanho n positivo que resulta no mesmo deslocamento
   negativo"""

# SOLUÇÃO 1
# Solução principal, utilizando estruturas de repetição
def shift(vetor: list, tamanho: int) -> list:
    """
    Função que recebe um vetor e um inteiro por parâmetro, e realiza um shift 
    no vetor do tamanho do número inteiro recebido. Números positivos indicam o 
    shift da direita para a esquerda, números negativos indicam o shift da 
    esquerda para a direita.
    
    Params:
        vetor (list): Vetor numérico
        tamanho (int): Número para realizar o deslocamento ('shift')

    Returns:
        novo_vetor (list): Novo vetor deslocado conforme o número fornecido
    """
    novo_vetor = vetor.copy()
    for i in range(len(vetor)):
        novo_vetor[i] = vetor[i-(tamanho%len(vetor))]
    return novo_vetor

In [16]:
# SOLUÇÃO 2
# Solução alternativa utilizando lambda e list comprehension (com estrutura de repetição)
shift2 = lambda vetor, tamanho: [vetor[i-(tamanho%len(vetor))] \
                                 for i in range(len(vetor))]

vetor = [0,1,2,3,4,5,6,7,8,9]

print(shift2(vetor, 1))  # Esperado [9,0,1,2,3,4,5,6,7,8]
print(shift2(vetor, 2))  # Esperado [8,9,0,1,2,3,4,5,6,7]
print(shift2(vetor, -1)) # Esperado [1,2,3,4,5,6,7,8,9,0]
print(shift2(vetor, -2)) # Esperado [2,3,4,5,6,7,8,9,0,1]

[9, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[8, 9, 0, 1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
[2, 3, 4, 5, 6, 7, 8, 9, 0, 1]


### Execute os campos a baixo para validar a função criada:

### Tamanho ``1`` - Resultado esperado: ``[9,0,1,2,3,4,5,6,7,8]``

In [17]:
vetor = [0,1,2,3,4,5,6,7,8,9]
ret = shift(vetor, 1)
print(ret)

[9, 0, 1, 2, 3, 4, 5, 6, 7, 8]


### Tamanho ``2`` - Resultado esperado: ``[8,9,0,1,2,3,4,5,6,7]``

In [18]:
vetor = [0,1,2,3,4,5,6,7,8,9]
ret = shift(vetor, 2)
print(ret)

[8, 9, 0, 1, 2, 3, 4, 5, 6, 7]


### Tamanho ``-1`` - Resultado esperado: ``[1,2,3,4,5,6,7,8,9,0]``

In [19]:
vetor = [0,1,2,3,4,5,6,7,8,9]
ret = shift(vetor, -1)
print(ret)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 0]


### Tamanho ``-2`` - Resultado esperado: ``[2,3,4,5,6,7,8,9,0,1]``

In [20]:
vetor = [0,1,2,3,4,5,6,7,8,9]
ret = shift(vetor, -2)
print(ret)

[2, 3, 4, 5, 6, 7, 8, 9, 0, 1]


## 5. Web Scraping

### 5.1 Desenvolva um algoritmo capaz de retornar os valores do mês atual, acumulado no ano e acumulado nos últimos 12 meses para os seguintes indicadores econômicos:
- Selic
- CDI
- Ibovespa
- Dólar Comercial (mercado)
- IPCA
- IGPM

    Os dados devem ser retirados do site __[Valor Econômico](https://www.valor.com.br/valor-data/tabela/5810/evolucao-das-aplicacoes-financeiras)__

In [21]:
# Desenvolva a Resposta do item 5.1 Aqui

# OBS: Precisa ter uma conta no site Valor Econômico para ver a tabela

### 5.2 Desenvolva um algoritmo capaz de retornar os valores das maiores altas e as maiores baixas do dia na bolsa de valores de São Paulo. Dados a serem retornados:
- Nome do papel (ex: PETR4)
- Último valor
- Variação
- Os dados podem ser retirados dos sites :
    - __[O Globo](https://oglobo.globo.com/economia/indicadores/)__
    - __[Infomoney](https://www.infomoney.com.br/)__
    - __[ADVFN](https://br.advfn.com/)__

In [22]:
# Desenvolva a Resposta do item 5.2 Aqui

res = requests.get('https://br.advfn.com/')

bs = BeautifulSoup(res.content, 'html.parser')
data = bs.find('div', {'class': 'main-wrap front-content'}) \
         .find('div', {'class': 'blocks'}) \
         .find('div', {'class': 'col1 ff-sidebar-sibling'}) \
         .find('div', {'class': 'wr-content-home block-first'}) \
         .find('div', {'class': 'bx-content-container toplist'}) \
         .find('div', {'class': 'col-ranking'}) \
         .find('table', {'class': 'table_element_class'}) \
         .find('tr', {'class': 'even first'})

papel = data.find('td', {'class': 'String Column2'}) \
            .find('a').text

var = data.find('td', {'class': 'PriceTextUp Numeric Column3'}) \
          .text

ultimo_val = data.find('td', {'class': 'Numeric Column4'}) \
                 .text

print(f"Nome do Papel: {papel}\n" \
      f"Último Valor: BRL {ultimo_val}\n" \
      f"Variação: {var}\n")

Nome do Papel: S2QU34
Último Valor: BRL 100,00
Variação: +84,88%



# Teste Python + SQL

1. Utilize um banco de dados(SQL) de sua preferência.
2. Se seu banco de dados tiver limite de espaço, adapte o dataset do python para que possa funcionar. Vamos analisar o código.
3. **Não esqueça de retirar os dados de acesso ao seu banco de dados: LOGIN e SENHA, ao enviar a resposta**

## 6. Gravar arquivos em uma tabela de banco de dados

A partir do arquivo ``receitas_candidatos_2014_brasil_english.csv`` executar um codigo em python que grave os dados do arquivo em uma tabela do banco de dados

Considere o seguinte cenario:

1. Estamos criando uma rotina para inserir registros do arquivo no banco de dados.
2. Ao executar a primeira vez o codigo, se a tabela não existir ela devera ser criada.
    - Ao Criar a tabela ou inserir dados, converter os registros de data/hora e númericos para o tipo correspondente no banco de dados
<br><br>
3. O nome da tabela deve ser ``TABELA_DOACOES``
4. O Arquivo não tem uma chave única, então vamos criar uma com o nome ``di_doacao``, para considerar um registro único, considere 3 colunas:
    - ``dat_donation_date``
    - ``id_donator_cpf_cnpj``
    - ``id_candidate_cpf``
    - Quando uma doação tiver a mesma chave (campos acima) vamos considerar o valor mais alto de doação para inserir na base de dados.
<br><br>
5. Em uma nova carga devemos analisar se um registro com a mesma chave já existe na tabela:
    - Caso não exista, vamos inserir o registro
    - Caso exista vamos analisar se o valor doado atual é maior que o valor. Caso seja maior atualizar o registro, caso contrario não efetuar nenhuma alteração
    
    **Obs**: Considerar que o arquivo sempre será uma extração full da origem, ou seja, sempre terá todos os dados que já foram processados e dados novos

In [23]:
# Desenvolva a Resposta do item 6 Aqui

engine = db.create_engine('sqlite:///receitas_candidatos_2014_brasil_english.sql', 
                          echo=False,
                          encoding="utf-8")

In [24]:
# Replace em 'np.nan' para uma string 'NaN',
# evitando erro no 'INSERT' do SQL
df.replace(np.nan, 'NaN', inplace=True)

In [25]:
"""
Converter os formatos atuais de data para datetime, e 
posteriormente converter novamente para 'str', alterando 
o formato para o correpondente a 'DATETIME' do SQL.

COLUNA 'dat_donation_date_time'

    Formato antigo
    --------------------------
    type: str
    Formato antigo: '%d/%m/%Y%H:%M:%S'
    Exemplo formato: 09/07/201617:15:01

    Formato novo
    --------------------------
    type: str
    Formato novo: 'YYYY-MM-DD HH:MI:SS'
    Exemplo de novo formato: 2016-07-09 17:15:01

COLUNA 'dat_donation_date'

    Formato antigo
    --------------------------
    type: str
    Formato antigo: '%d/%m/%Y%H:%M:%S'
    Exemplo formato antigo: 02/10/201400:00:00
    
    Formato novo
    --------------------------
    type: str
    Formato novo: 'YYYY-MM-DD HH:MI:SS'
    Exemplo de novo formato: 2014-10-02 00:00:00
    """

df['dat_donation_date_time'] = pd.to_datetime(df['dat_donation_date_time'], 
                                         format='%d/%m/%Y%H:%M:%S')

df['dat_donation_date_time'] = df['dat_donation_date_time'] \
                               .apply(lambda x: str(x))

df['dat_donation_date'] = pd.to_datetime(df['dat_donation_date'], 
                                         format='%d/%m/%Y%H:%M:%S')

df['dat_donation_date'] = df['dat_donation_date'] \
                          .apply(lambda x: str(x))

In [26]:
"""Como alternativa, podemos criar uma chave composta pelas
   colunas mencionadas, porém teríamos um problema relacionado
   às linhas já duplicadas (que possuem as três colunas repetidas)"""

create_table = """CREATE TABLE TABELA_DOACOES (
					id_election BIGINT, 
					cat_election_description TEXT, 
					dat_donation_date_time TEXT, 
					id_accountant_cnpj BIGINT, 
					id_candidate_seq BIGINT, 
					cat_election_state TEXT, 
					cat_party TEXT, 
					id_candidate_num BIGINT, 
					cat_political_office TEXT, 
					cat_candidate_name TEXT, 
				    id_candidate_cpf BIGINT, 
					id_receipt_num TEXT, 
					id_document_num TEXT, 
				    id_donator_cpf_cnpj FLOAT, 
					cat_donator_name TEXT, 
					cat_donator_name2 TEXT, 
					cat_donator_state TEXT, 
					id_donator_party FLOAT, 
					id_donator_number BIGINT, 
					id_donator_economic_sector FLOAT, 
					cat_donator_economic_sector TEXT, 
				    dat_donation_date DATETIME, 
					num_donation_ammount FLOAT, 
					cat_donation_type TEXT, 
					cat_donation_source TEXT, 
					cat_donation_method TEXT, 
					cat_donation_description TEXT, 
				    id_original_donator_cpf_cnpj FLOAT, 
					cat_original_donator_name TEXT, 
					cat_original_donator_type TEXT, 
					cat_original_donator_economic_sector TEXT,
					cat_original_donator_name2 TEXT,
					di_doacao INT NOT NULL,
					PRIMARY KEY (di_doacao))"""

tb2 = engine.execute(create_table)

In [27]:
# Para fins de otimização de tempo, vamos utilizar apenas as 200 primeiras
# linhas da base de dados

# Lista com as linhas a serem adicionadas na nova tabela
# Contém os registros até o 200, para otimizar o tempo de execução
rows = []
for i in range(200):
    row = list(df.loc[i][1:])
    row.append(i+1)
    row = tuple(row)
    rows.append(row)

def tb_insert_th(a, b):
    """
    Função que insere os dados na tabela criada, variando da posição a até b
    em relação à lista contendo os dados a serem inseridos, comparando os 
    valores doados em caso de colisão da chave primária, bem como substituindo 
    o valor doado pelo maior encontrado.

    Params:
        a (int): index de início da lista de linhas a serem inseridas
        b (int): index final da lista de linhas a serem inseridas

    Returns:
        None: Insere os dados na tabela TABELA_DOACOES criada.
    """

    for i in range(a, b):
        id_candidate_cpf = str(rows[i][10])
        id_original_donator_cpf_cnpj = str(rows[i][13])
        dat_donation_date = str(rows[i][21])

        temp = f"""SELECT num_donation_ammount 
                FROM TABELA_DOACOES
                WHERE dat_donation_date = '{dat_donation_date}'
                        AND id_donator_cpf_cnpj = '{id_original_donator_cpf_cnpj}'
                        AND id_candidate_cpf = '{id_candidate_cpf}' """

        duplicates = engine.execute(temp).fetchall()

        if len(duplicates) == 0:
            ins = f"""INSERT INTO TABELA_DOACOES VALUES """ + \
                      str(rows[i])
            engine.execute(ins)
        else:
            max_ammount = max(duplicates)
            upd = f"""UPDATE TABELA_DOACOES 
                      SET num_donation_ammount = {max_ammount[0]}
                      WHERE dat_donation_date = '{dat_donation_date}'
                            AND id_donator_cpf_cnpj = '{id_original_donator_cpf_cnpj}'
                            AND id_candidate_cpf = '{id_candidate_cpf}' """

            engine.execute(upd)

In [28]:
"""Possível solução alternativa, que obtém a linha com a maior doação dentro
   das linhas com mesmos campos de chave, deleta todas as linhas e insere
   esta com maior valor. Foi descartada, pois da erro no SQLite, devido às
   suas limitações de concorrência"""

# for i in range(200):
#     id_candidate_cpf = str(rows[i][10])
#     id_original_donator_cpf_cnpj = str(rows[i][13])
#     dat_donation_date = str(rows[i][21])
    
#     temp = f"""SELECT num_donation_ammount 
#                 FROM TABELA_DOACOES
#                 WHERE dat_donation_date = '{dat_donation_date}'
#                         AND id_donator_cpf_cnpj = '{id_original_donator_cpf_cnpj}'
#                         AND id_candidate_cpf = '{id_candidate_cpf}' """

#     duplicates = engine.execute(temp).fetchall()

#     if len(duplicates) == 0:
#         ins = f"""INSERT INTO TABELA_DOACOES VALUES """ + \
#                     str(rows[i])
#         engine.execute(ins)

#     else:
#         max_ammount = max(duplicates)

#         max_row = f"""SELECT * 
#                  FROM TABELA_DOACOES
#                  WHERE num_donation_ammount = (SELECT 
#                                                MAX(num_donation_ammount)
#                                                FROM ({temp}))"""
#         max_row_res = engine.execute(max_row)

#         dele = f"""DELETE FROM TABELA_DOACOES
#                     WHERE dat_donation_date = '{dat_donation_date}'
#                         AND id_donator_cpf_cnpj = '{id_original_donator_cpf_cnpj}'
#                         AND id_candidate_cpf = '{id_candidate_cpf}' """
#         engine.execute(dele)

#         ins2 = f"""INSERT INTO TABELA_DOACOES VALUES """ + \
#                    str(list(max_row_res)[0])
#         engine.execute(ins)
pass

In [29]:
# Leva um certo tempo pra rodar devido ao número de linhas
# O ideal é usar PySpark para essa task

# Criando Threads para acelerar o processo, principalmente em
# maiores quantidades de dados
# 427489 -> Tamanho total dos dados (em linhas)

th1 = Thread(target=tb_insert_th(0, 100))
th2 = Thread(target=tb_insert_th(100, 200))

th1.start()
th2.start()

In [30]:
# Consultando 10 registros da nova tabela
query_validation = """SELECT *
                      FROM TABELA_DOACOES
                      LIMIT 10;"""

dados_tabela_nova = engine.execute(query_validation) \
                          .fetchall()

In [31]:
# Avaliando se existe alguma PK duplicada (mesmos campos 
# nas 3 colunas) em linhas distintas
id_candidate_cpf_list = [row[10] \
                         for row \
                         in dados_tabela_nova]

id_original_donator_cpf_cnpj_list = [row[13] \
                                     for row \
                                     in dados_tabela_nova]
                                     
dat_donation_date_list = [row[21] \
                          for row \
                          in dados_tabela_nova]

unique_pks = []
duplicates = []

for x, y, z in zip(id_candidate_cpf_list, 
                   id_original_donator_cpf_cnpj_list,
                   dat_donation_date_list):
    el = (x, y, z)
    if el not in unique_pks:
        unique_pks.append(el)
    else:
        duplicates.append(el)

if len(duplicates) == 0: 
    print("Nenhuma PK duplicada!")

Nenhuma PK duplicada!


In [32]:
# Contando, considerando os 200 registros inseridos,
# um número reduzido (pois existiam linhas em que as colunas
# da pk eram iguais)
query_count = """SELECT COUNT(*) 
                 FROM TABELA_DOACOES"""

engine.execute(query_count).fetchall()

[(179,)]

In [33]:
# Convertendo o pandas df para uma tabela sql
# Será usada uma tabela gerada pelo pandas para os itens 7 a 10, 
# pois possui todos os registros, uma vez que a tabela criada 
# acima foi populada com apenas 200 linhas
df.to_sql('TABELA_DOACOES2', con=engine, if_exists='replace')

# Teste SQL

1. Utilize um banco de dados (SQL) de sua preferência.

2. De Preferência utilize a tabela criada no item anterior, caso não tenha a tabela faça uma carga do arquivo ``receitas_candidatos_2014_brasil_english.csv`` em uma tabela chamada ``TABELA_DOACOES``. Se seu banco de dados tiver limite de espaço, adapte a quantidade de linhas para inserir na tabela. Vamos analisar as instruções SQL.
    
**Obs**: Somente cole a instrução SQL nas células.

## 7. Crie uma *query* que responda:

Quais são os 10 candidatos que receberam mais doações?
        
 

```sql
-- Cole a instrução SQL aqui
SELECT cat_candidate_name, 
       id_candidate_seq, 
       COUNT(num_donation_ammount)
FROM TABELA_DOACOES
GROUP BY id_candidate_seq
ORDER BY COUNT(num_donation_ammount) DESC
LIMIT 10;
```

In [34]:
# Executando a query
query1 = """SELECT cat_candidate_name, 
                id_candidate_seq, 
                COUNT(num_donation_ammount)
            FROM TABELA_DOACOES2
            GROUP BY id_candidate_seq
            ORDER BY COUNT(num_donation_ammount) DESC
            LIMIT 10;"""

print("(Nome, Id de candidato, Soma das doações)")
engine.execute(query1).fetchall()

(Nome, Id de candidato, Soma das doações)


[('LUIZ FERNANDO DE SOUZA', 190000001649, 6530),
 ('AÉCIO NEVES DA CUNHA', 280000000085, 3158),
 ('DILMA VANA ROUSSEFF', 280000000083, 3017),
 ('CONFUCIO AIRES MOURA', 220000000187, 2221),
 ('CARLOS ALBERTO RICHA', 160000000558, 1429),
 ('JOÃO PIMENTA DA VEIGA FILHO', 130000000198, 1357),
 ('EUNÍCIO LOPES DE OLIVEIRA', 60000000603, 1310),
 ('ANTONIO AUGUSTO JUNHO ANASTASIA', 130000001777, 1304),
 ('JOSÉ WELLINGTON BARROSO DE ARAÚJO DIAS', 180000000212, 1202),
 ('MARCIA CRISTINA ARAUJO JEOVANI', 190000001783, 1109)]

In [35]:
# Validando os resultados através do pandas
query1_validation = df.groupby('id_candidate_seq') \
                      .agg('count')['num_donation_ammount']

query1_validation.sort_values(ascending=False)[:10]

id_candidate_seq
190000001649    6530
280000000085    3158
280000000083    3017
220000000187    2221
160000000558    1429
130000000198    1357
60000000603     1310
130000001777    1304
180000000212    1202
190000001783    1109
Name: num_donation_ammount, dtype: int64

## 8. Crie uma *query* que responda:

Para o posto (``cat_political_office``) de "Deputado Federal" qual o valor que cada candidato recebeu, o valor médio de doções e a relação entre os dois, ordene pelos candidatos com mais doações relativas.
 

```sql
-- Cole a instrução SQL aqui
SELECT cat_candidate_name,
id_candidate_seq, 
SUM(num_donation_ammount) AS sum,
(SELECT AVG(num_donation_ammount)
 FROM TABELA_DOACOES
 WHERE cat_political_office = 'Deputado Federal') AS avg,
SUM(num_donation_ammount) / (SELECT AVG(num_donation_ammount)
                             FROM TABELA_DOACOES
                             WHERE cat_political_office = 'Deputado Federal') 
                             AS ratio
FROM TABELA_DOACOES
WHERE cat_political_office = 'Deputado Federal'
GROUP BY id_candidate_seq
ORDER BY ratio DESC
LIMIT 10;
```

In [36]:
# Executando a query
query2 = """SELECT cat_candidate_name,
                   id_candidate_seq, 
                   SUM(num_donation_ammount) AS sum,
                   (SELECT AVG(num_donation_ammount)
                    FROM TABELA_DOACOES2
                    WHERE cat_political_office = 'Deputado Federal') AS avg,
                   SUM(num_donation_ammount) / (SELECT AVG(num_donation_ammount)
                                                 FROM TABELA_DOACOES2
                                                 WHERE cat_political_office = 'Deputado Federal')
                                                 AS ratio
            FROM TABELA_DOACOES2
            WHERE cat_political_office = 'Deputado Federal'
            GROUP BY id_candidate_seq
            ORDER BY ratio DESC
            LIMIT 10;"""

print("(Nome, Id de candidato, Soma das doações, Valor médio, Razão)")
engine.execute(query2).fetchall()

(Nome, Id de candidato, Soma das doações, Valor médio, Razão)


[('EDUARDO COSENTINO DA CUNHA', 190000001421, 6832480.98, 9382.194128971398, 728.2391395954913),
 ('MARCO ANTÔNIO NEVES CABRAL', 190000001459, 6789094.5, 9382.194128971398, 723.6147969946462),
 ('IRACEMA MARIA PORTELLA NUNES NOGUEIRA LIMA', 180000000261, 6723539.5, 9382.194128971398, 716.6276254334043),
 ('CARLOS ALBERTO ROLIM ZARATTINI', 250000000618, 6244292.849999999, 9382.194128971398, 665.5471805596269),
 ('BENITO DA GAMA SANTOS', 50000000121, 5979284.139999996, 9382.194128971398, 637.3012600044682),
 ('SERGIO ZVEITER', 190000001486, 5720550.5, 9382.194128971398, 609.7241670085933),
 ('ARTHUR VIRGILIO DO CARMO RIBEIRO BISNETO', 40000000353, 5637809.059999932, 9382.194128971398, 600.9051808671138),
 ('MARCOS ABRAO RORIZ SOARES DE CARVALHO', 90000000711, 5526035.899999999, 9382.194128971398, 588.9918524427119),
 ("ALEXANDRE BALDY DE SANT'ANNA BRAGA", 90000000744, 5484143.06, 9382.194128971398, 584.5267092763987),
 ('CRISTIANE BRASIL FRANCISCO', 190000001460, 5365728.32, 9382.1941289

In [37]:
# Validando os resultados através do pandas
deputados = df[df['cat_political_office'] == 'Deputado Federal']

deputados_donations = deputados.groupby('id_candidate_seq') \
                               .agg('sum')['num_donation_ammount']

deputados_mean = deputados['num_donation_ammount'] \
                 .mean()

deputados_ratio = lambda x: x / deputados_mean
deputados_donations_ratio = deputados_donations \
                            .apply(deputados_ratio)

query2_validation = pd.DataFrame()
query2_validation['sum_donation_ammount'] = deputados_donations
query2_validation['mean'] = [deputados_mean for i in range(len(deputados_donations))]
query2_validation['ratio'] = deputados_donations_ratio

query2_validation.sort_values(by='ratio', 
                              ascending=False).head()

,sum_donation_ammount,mean,ratio
id_candidate_seq,,,
190000001421,6832480.98,9382.194129,728.239140
190000001459,6789094.50,9382.194129,723.614797
180000000261,6723539.50,9382.194129,716.627625
250000000618,6244292.85,9382.194129,665.547181
50000000121,5979284.14,9382.194129,637.301260


## 9. Crie uma *query* que responda:

Liste todos os candidatos, o seu valor recebido em doação, o valor médio de doações pelo posto correspondente ao do candidato e a relação entre o Recebido pelo candidato e a média do posto concorrido.

```sql
-- Cole a instrução SQL aqui
-- Pendente: Reduzir o tamanho da query, não repetindo duas vezes o 'CASE'
SELECT cat_political_office,
            cat_candidate_name,
            id_candidate_seq,
            SUM(num_donation_ammount) AS sum,
            CASE
                WHEN cat_political_office = 'Governador' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Governador')

                WHEN cat_political_office = 'Deputado Federal' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Deputado Federal')

                WHEN cat_political_office = 'Senador' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Senador')

                WHEN cat_political_office = 'Deputado Estadual' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Deputado Estadual')

                WHEN cat_political_office = 'Deputado Distrital' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Deputado Distrital')

                WHEN cat_political_office = 'Presidente' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Presidente')
            END AS office_avg,

            SUM(num_donation_ammount) / CASE
                WHEN cat_political_office = 'Governador' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Governador')

                WHEN cat_political_office = 'Deputado Federal' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Deputado Federal')

                WHEN cat_political_office = 'Senador' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Senador')

                WHEN cat_political_office = 'Deputado Estadual' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Deputado Estadual')

                WHEN cat_political_office = 'Deputado Distrital' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Deputado Distrital')

                WHEN cat_political_office = 'Presidente' 
                    THEN (SELECT AVG(num_donation_ammount)
                            FROM TABELA_DOACOES
                            GROUP BY cat_political_office
                            HAVING cat_political_office = 'Presidente')
            END AS office_avg    
    FROM TABELA_DOACOES
    GROUP BY id_candidate_seq
    LIMIT 10;
```

In [38]:
# Armazenando o trecho do 'CASE' para reduzir o tamanho da query no python
# Criando o trecho de forma dinâmica
# Para ver o trecho completo, visializar na query acima
case_statement = """CASE """
offices = ['Governador', 'Deputado Federal', 
           'Senador', 'Deputado Estadual', 
           'Deputado Distrital', 'Presidente']

for office in offices:
    case = f"""WHEN cat_political_office = '{office}'
               THEN (SELECT AVG(num_donation_ammount)
                   FROM TABELA_DOACOES2
                   GROUP BY cat_political_office
                   HAVING cat_political_office = '{office}') """
    case_statement += case
case_statement += "END AS office_avg"

In [39]:
# Executando a query
# Utilizando o trecho 'CASE' armazenado na variável 'case_statement'
query3 = """SELECT cat_political_office,
                   cat_candidate_name,
                   id_candidate_seq,
                   SUM(num_donation_ammount) AS sum, """ + \
                   case_statement + """,

                   SUM(num_donation_ammount) / """ + \
                   case_statement + \
               """ FROM TABELA_DOACOES2
            GROUP BY id_candidate_seq
            LIMIT 10;"""

print("(Cargo, Nome, Id de candidato, Soma das doações, Média do cargo, Razão)")
table = engine.execute(query3).fetchall()
table

(Cargo, Nome, Id de candidato, Soma das doações, Média do cargo, Razão)


[('Senador', 'GLADSON DE LIMA CAMELI', 10000000001, 4914918.29, 23854.841326077607, 206.03441552248412),
 ('Governador', 'MÁRCIO MIGUEL BITTAR', 10000000003, 2150861.6799999997, 30894.562022189435, 69.61942617782327),
 ('Deputado Estadual', 'HENRY ANTONIO SILVA NOGUEIRA', 10000000021, 31065.81, 5186.874991018515, 5.98931149368221),
 ('Deputado Estadual', 'ZENILDO VIEIRA DA SILVA', 10000000022, 16530.0, 5186.874991018515, 3.1868899922637435),
 ('Deputado Estadual', 'JOSÉ ALVES DA SILVA', 10000000023, 1141.55, 5186.874991018515, 0.22008434789284187),
 ('Deputado Estadual', 'MARCOS GLAY RIBEIRO DE SOUZA', 10000000024, 3506.45, 5186.874991018515, 0.6760236184738779),
 ('Deputado Estadual', 'JOSÉ ALTANIZIO TAUMATURGO SÁ', 10000000025, 32454.889999999996, 5186.874991018515, 6.257118217847588),
 ('Deputado Estadual', 'AURELIANO ALVES RABELO', 10000000026, 3897.35, 5186.874991018515, 0.7513869153871204),
 ('Deputado Estadual', 'JOSE AUGUSTO PINHEIRO DA SILVEIRA', 10000000027, 19697.29, 5186.87

In [40]:
# Validando os resultados através do pandas
groupby_sum = df.sort_values(by='id_candidate_seq', ascending=True) \
                .groupby('id_candidate_seq') \
                .agg('sum')['num_donation_ammount']

groupby_sum.head()

id_candidate_seq
10000000001    4914918.29
10000000003    2150861.68
10000000021      31065.81
10000000022      16530.00
10000000023       1141.55
Name: num_donation_ammount, dtype: float64

In [41]:
# Validando os resultados através do pandas
groupby_office = df.groupby('cat_political_office') \
                   .agg('mean')['num_donation_ammount']

groupby_office

cat_political_office
Deputado Distrital     2821.138642
Deputado Estadual      5186.874991
Deputado Federal       9382.194129
Governador            30894.562022
Presidente            84480.690210
Senador               23854.841326
Name: num_donation_ammount, dtype: float64

In [42]:
# Validando os resultados através do pandas
ratio_sum_office = lambda x: groupby_sum[x['id_candidate_seq']] / \
                             groupby_office[x['cat_political_office']]

ratio_sum_office_col = df.sort_values(by='id_candidate_seq', ascending=True) \
                         .apply(ratio_sum_office, axis=1)

ratio_sum_office_col.head()

149492    206.034416
180352    206.034416
318837    206.034416
356088    206.034416
305712    206.034416
dtype: float64

## 10. Crie uma *query* que responda:

Crie uma tabela com base no resultado anterior, filtrando somente candidatos que receberam mais do que 0,1% da média total do seu posto. 

```sql
-- Cole a instrução SQL aqui
SELECT cat_candidate_name, 
       id_candidate_seq,
       sum
FROM QUERY_3
WHERE sum > 0.1 * office_avg
LIMIT 10;
```

In [43]:
# Criando uma tabela com base na query anterior (Item 9)
# Nome da nova tabela: 'QUERY_3'
create_table = """CREATE TABLE QUERY_3 AS """ + \
                  query3

engine.execute(create_table)

In [44]:
# Executando a query
# Selecionando a partir da nova tabela criada
query4 = """SELECT cat_candidate_name, 
                   id_candidate_seq,
                   sum
            FROM QUERY_3
            WHERE sum > 0.1 * office_avg
            LIMIT 10;"""

print("(Nome, Id de candidato, Soma das doações)")
engine.execute(query4).fetchall()

(Nome, Id de candidato, Soma das doações)


[('GLADSON DE LIMA CAMELI', 10000000001, 4914918.29),
 ('MÁRCIO MIGUEL BITTAR', 10000000003, 2150861.6799999997),
 ('HENRY ANTONIO SILVA NOGUEIRA', 10000000021, 31065.81),
 ('ZENILDO VIEIRA DA SILVA', 10000000022, 16530.0),
 ('JOSÉ ALVES DA SILVA', 10000000023, 1141.55),
 ('MARCOS GLAY RIBEIRO DE SOUZA', 10000000024, 3506.45),
 ('JOSÉ ALTANIZIO TAUMATURGO SÁ', 10000000025, 32454.889999999996),
 ('AURELIANO ALVES RABELO', 10000000026, 3897.35),
 ('JOSE AUGUSTO PINHEIRO DA SILVEIRA', 10000000027, 19697.29),
 ('GILBERTO CABRAL DE MOURA', 10000000028, 25000.0)]